# Almgren-Chriss Optimal Execution - Dynamic Programming

## Syllabus

1. Almgren-Chriss framework
2. Dynamic programming
3. Vanilla implementation
4. From market orders to limit orders

## 1. Almgren-Chriss framework

The **Almgren-Chriss framework** (1999) is a mathematical model that looks to:

- Estimate the optimal pace to build/unwind a trading position, issues often faced by brokers and traders (but has not seen as much interest from statisticians, economists and econophysicists in the past decades);
- Allows one to graphically represent the trade-off between slow/fast execution;
- Ease to choose one's own prefered approach to model market impact costs;
- Avoids the temptation of modelling the market as a physical system fully enclosed in its own data (*physics envy*);
- Designed from a bottom-top approach: the modelling is based on the execution process itself;
- Provides a multitude of modelling approaches to compute the slow/fast execution trade-off (e.g. KKT nonlinear optimization, dynamic programming, stochastic control, reinforcement learning, etc.).

We will present 2 approaches to solve the Almgren-Chriss model:

- Quadratic Programming (original method by Almgren-Chriss)
- Dynamic Programming

This notebook will focus on the **dynamic programming** approach.


## 2. Dynamic programming

### 2.1. Market impact functions

In [148]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

Utilities for the implementation of Almgren-Chriss. For the market impact functions, we will follow the model assumptions made by Almgren, Thum, Hauptmann and Li (2005). Since we cannot estimate both permanent and temporary impact functions, we need to make assumptions on the structure of both functions.

- **Temporary market impact function**: Almgren and al. (2005) through empirical evidence indicate that this function should be concave, where $0<\beta<1$. The authors choose $\beta=0.5$.

$$ h \left(\frac{n_k}{\tau}\right) = \epsilon \, sign(n_k) + \eta \frac{n_k}{\tau} $$

We could also used:

$$ h \left(\frac{n_k}{\tau}\right) = \eta |v|^{\beta} $$

- **Permanent market impact function**: Almgren and al. (2005) take $\alpha=1$ for practical simplification, to make the model free of arbitrage and have permanent impact independent of trading.

$$ g(v) = \gamma |v|^{\alpha} $$

- **Hamiltonian equation**

$$ H(x,n) = \psi n g \left(\frac{n}{\tau}\right) + \psi (x-n) \tau h \left(\frac{n}{\tau}\right) + \frac{1}{2} \psi^2 (x-n)^2 \sigma^2 \tau $$

In [162]:
# Utilities
def h(v, tau=1.0, eta = 2.5*10**(-6)):
    """
    Temporary market impact function.
    
    Inputs
    - u/tau, speed of trading
    """
    epsilon = 0.0625
    return epsilon * np.sign(v) + eta * (v/tau)

def g(v, gamma=2.5*10**(-7)):
    """
    Permanent market impact function.
    """
    return gamma * v

def H(x, n, psi, sigma=0.3, tau=0.5):
    """
    Hamiltonian equation. To be minimized through dynamic programming.
    """
    # gamma = 0.1
    res = psi * n * g(n/tau) + psi*(x-n)*tau*h(n,tau) + 0.5*(psi**2)*(sigma**2)*tau*((x-n)**2)
    return res

### 2.2. Dynamic programming: a simplified version

Bellman equation with change of variable (made in our to solve the curse of dimensionality):

- **Terminal condition**

$$ u(x,T-1) = \exp \left[\psi x g \left(\frac{x}{\tau} \right) \right] $$

- **Backward induction**

$$ u(x,t) = \min_{0<n<x} u(x-n,t+1) \, \exp \left[\psi n g \left(\frac{n}{\tau}\right) + \psi (x-n) \tau h \left(\frac{n}{\tau}\right) + \frac{1}{2} \psi^2 (x-n)^2 \sigma^2 \tau \right] $$

Python implementation:

In [163]:
def dynamic_programming(nb_T, X_total, psi, plot='True'):
    
    """
    MODEL
    - Bellman equation and value iteration for solving the Markov
      Decision Process of the Almgren-Chriss model.
    
    INPUTS
    - nb_T,       number of time steps
    - X_total,    number of shares to be liquidated
    - psi,        risk aversion
    """
    
    ### Initialization
    u = np.zeros(shape=(nb_T, X_total+1), dtype="float64")      # value function
    b = np.zeros(shape=(nb_T, X_total+1), dtype="int")          # best move
    inventoryforX = np.zeros(shape=(nb_T,1), dtype="int")       # evolution of inventory
    inventoryforX[0] = X_total
    N = []                                                      # optimal selling trajectory
    tau = 0.5
    
    ### Market microstructure: volatility, correlation
    ### [insert dynamic volatility data]
    
    ### Terminal condition
    for x in range(X_total+1):
        u[nb_T - 1, x] = np.exp(x * h(x/tau))
        b[nb_T - 1, x] = x
    
    ### Backwards induction
    for t in range(nb_T-2, -1, -1):
        for x in range(X_total+1):
            
            best_value = u[t+1,0] * np.exp(H(x, x, psi))
            best_n = x
            
            for n in range(x):
                # We compute the utility function if we sell n shares
                current_value = u[t+1,x-n] * np.exp(H(x, n, psi))
                
                if current_value < best_value:
                    best_value = current_value
                    best_n = n   # nb of shares to liquidate
               
            u[t,x] = best_value
            b[t,x] = best_n
    
    ### Optimal trajectory
    for t in range(1, nb_T):
        inventoryforX[t] = inventoryforX[t-1] - b[t,inventoryforX[t-1]]
        N.append(b[t,inventoryforX[t-1]])
    
    N = np.asarray(N)
    
    ### Plot results
    if plot=='True':
        plt.figure(figsize=(7,5))
        plt.plot(inventoryforX, marker='o', color='blue')
        plt.xlabel('Day')
        plt.ylabel('Number of shares')
        plt.grid(True)
        plt.show()
    
    return u, b, inventoryforX, N

## 3. Vanilla implementation

### 3.1. Risk aversion $\psi$

- $\psi = 0.01$

In [ ]:
u, b, eps, N = dynamic_programming(nb_T=5, X_total=10000, psi=0.01, plot='True')
print(np.matrix(u))
print(np.matrix(b))

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in exp
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in double_scalars


## 4. From market orders to limit orders